In [ ]:
import os
import sys
from quetzal.os.parallel_call import parallel_call_notebook, parallel_call_notebooks

In [ ]:
if not os.path.exists('log/'):
    os.makedirs('log/')

# Launcher

This notebook automatically launches all operations for a complete model run. Network preparation steps (`prep1X` and `prep2X`) are not covered here, as they take several hours and are not needed for simulation of scenarios that do not alter transport networks.

Detailed explainations of certain steps can be found in the corresponding notebook. All ASSUMPTIONS are gathered in the `input/parameters.xls` file. This file also includes scenarios as columns, which can be run by putting them in the `scenarios` list below.

In [ ]:
kwargs = {'workers':4, 'errout_suffix':True, 'sleep':1,'stdout_path':r'log/out.txt', 'stderr_path':r'log/err.txt'}

In [ ]:
scenarios = ['base'] #['Avoid', 'Shift']

# Socio-demographics

First step in every scenario

In [ ]:
# Must be ran for every scenario to apply socio-demographic developments
parallel_call_notebook('prep10_zones_disagg.ipynb', arg_list=scenarios, **kwargs)

In [ ]:
# OPTIONAL: Only if the car ownership module is installed
#parallel_call_notebook('../car_ownership/carownership_module_for_quetzal_germany.ipynb', arg_list=scenarios, **kwargs)

# Pathfinders and level of service

Create the shortest-paths-stack with network performance attributes for all modes based on shortest paths.

If scenario assumptions in `parameters.xls` are changed, the corresponding notebook should be ran (e.g. `prep31_prices_car` in case of petrol price changes).

Note: All upcoming steps should be run on the same computer, as intermediate model files are pickled and Python's pickle protocols are not stable across different versions.

In [ ]:
# Road
parallel_call_notebook('prep30_pathfinder_road.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep31_prices_car.ipynb', arg_list=scenarios, **kwargs)

In [ ]:
# PT
parallel_call_notebook('prep30_pathfinder_pt.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep31_prices_pt.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep33_non-motor.ipynb', arg_list=scenarios, **kwargs)
# terminates with error from 'assert manual' if there is a saved versions of footpaths
# so that it doesn't have to compute them again

# 4-step modelling

Run these steps in order. Mode choice generates the composite cost, which is an input for generation and distribution. The last step, assignment, also computes total pkm, when inner-zonal traffic is generated before.

In [ ]:
# Mode choice probabilities and composite cost
parallel_call_notebook('model_mode.ipynb', arg_list=scenarios, **kwargs)

In [ ]:
# Trip generation
#parallel_call_notebook('model_generation_exo.ipynb', arg_list=scenarios, **kwargs) # fixed probabilities from MiD2017
parallel_call_notebook('model_generation_endo.ipynb', arg_list=scenarios, **kwargs) # Logit model calibrated with MiD2017

In [ ]:
# Generation of the OD matrix
# Calculates compulsory trips with doubly constrained method
parallel_call_notebook('model_volumes_distribution.ipynb', arg_list=scenarios, **kwargs)
# Simulates choice models based on zone and network attributes
parallel_call_notebook('model_volumes_endo.ipynb', arg_list=scenarios, **kwargs)

In [ ]:
# Generation of inner-zonal volumes, pkm, prices, and times
parallel_call_notebook('model_inner-zonal.ipynb', arg_list=scenarios, **kwargs)

In [ ]:
# Assignment and output plots
parallel_call_notebook('model_assignment.ipynb', arg_list=scenarios, **kwargs)

# Equilibration

Demand-supply equilibration happens iteratively for road traffic. It can be used to refine results after running the assignment step once. If so, a new model folder `<scenario name>_equilibrium` is created from which the modelling steps take their road LoS table and save the equilibrated results.

**NOTE: Currently inactive due to missing link capacity data.**

In [ ]:
# Run the equilibrium road pathfinder to create the new LoS table
parallel_call_notebook('model_assignment_equilibrium.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep31_prices_car.ipynb', arg_list=scenarios, **kwargs)
# Rename the scenarios
scenarios = [str(s)+'_equilibrium' for s in scenarios]

In [ ]:
# Demand model
parallel_call_notebook('model_mode.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('model_generation_endo.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('model_destination.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('model_volumes_endo.ipynb', arg_list=scenarios, **kwargs)
# Supply model
parallel_call_notebook('model_assignment.ipynb', arg_list=scenarios, **kwargs)